In [10]:

import tensorflow.keras as kr
import nibabel as nib
import numpy as np
import os
from glob import glob
import matplotlib.pylab as plt

folder_path = [os.path.join(r"BraTS2021_Training_Data", i) for i in os.listdir(r"BraTS2021_Training_Data")]
imgs = []


for i in folder_path:
    imgs.append(glob(i+"/*"))
print("Done")
folder_path = folder_path[1:]
imgs = imgs[1:]
print(imgs[0])



#input directory and output directory
outputpath = []
inputpath = []
for j in imgs:
    for i in j:
        if "seg.nii.gz" in i.split("_"):
            outputpath.append(i)
for j in imgs:
    temppath = []
    for i in j:
        if "seg.nii.gz" not in i.split("_"):
            temppath.append(i)
    inputpath.append(temppath)
    
x_train_path = inputpath[:1126].copy()
y_train_path = outputpath[:1126].copy()
x_eval_path = inputpath[1126:1189].copy()
y_eval_path = outputpath[1126:1189].copy()
x_test_path = inputpath[1189:].copy()
y_test_path = outputpath[1189:].copy()  

parition = {"train": x_train_path, "validation": x_eval_path}
label = {"train": y_train_path, "validation": y_eval_path}
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------OLD-------------------------------------------------------------------------------
# def prepoutput():
#     index = 0
#     outputimg = np.array((nib.load(outputpath[0]).get_fdata().astype(np.float32)[:, :, 75]))
#     outputimg = np.expand_dims(outputimg, axis=2)
#     outputimg = tf.image.resize(outputimg, (256, 256))
#     for i in range(1,len(outputpath)):
#         index+=1
#         print(index)
#         x = nib.load(outputpath[i]).get_fdata().astype(np.float32)[:, :, 75]
#         x = np.expand_dims(x, axis=2)
#         x = tf.image.resize(x, (256, 256))
#         outputimg = np.dstack((outputimg,x),)
#     return outputimg
# outputimg = prepoutput()
# outputimg = outputimg.transpose((2, 0, 1))
# np.save('output_dataset.npy', outputimg)

#


# inputimg = nib.load(inputpath[0][0]).get_fdata().astype(np.float32)[:, :, 75]
# for i in range(1,len(inputpath[0])):
#     inputimg = np.dstack((inputimg,nib.load(inputpath[0][i]).get_fdata().astype(np.float32)[:, :, 75]))
# inputimg = np.expand_dims(inputimg, axis=0)

# for i in range(1,len(inputpath)):
#     print(i)
    
#     temp = nib.load(inputpath[i][0]).get_fdata().astype(np.float32)[:, :, 75]
#     print("\t1")
#     print(f"\t{inputpath[i][0]}")
#     for channels in range(1,len(inputpath[i])):
#         print(f"\t{channels+1}")
#         print(f"\t{inputpath[i][channels]}")
#         temp = np.dstack((temp,nib.load(inputpath[i][channels]).get_fdata().astype(np.float32)[:, :, 75]))
        
#     temp = np.expand_dims(temp, axis=0)
#     inputimg = np.concatenate((inputimg,temp), axis=0)
#     print(inputimg.shape)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------------------------------------------------

import numpy as np
import keras
import random
import time
import os
import nibabel as nib

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(240,240,140), n_channels=4,
                 n_classes=10, shuffle=True, real_batchsize_custom=2, frames_chunk=18,
                 y_VolSlice_start=0, y_VolSlice_end=None):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.real_batchsize_custom = real_batchsize_custom
        self.frames_chunk = frames_chunk
        self.y_VolSlice_start = y_VolSlice_start
        self.y_VolSlice_end = y_VolSlice_end if y_VolSlice_end is not None else dim[2]

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        sliced_dim = (self.dim[0], self.dim[1], self.y_VolSlice_end - self.y_VolSlice_start)
        X = np.empty((self.real_batchsize_custom, *sliced_dim, 4))
        y = np.empty((self.real_batchsize_custom,*sliced_dim, 4), dtype=float)

        initial_point = 0
        final_point = self.frames_chunk
        for x in range(self.real_batchsize_custom):    
            indexes_orig = indexes[initial_point:final_point]
            list_IDs_temp = [self.list_IDs[k] for k in indexes_orig]
            X_in, y_in = self.__data_generation(list_IDs_temp)
            X[x] = X_in
            y[x] = y_in
            initial_point = final_point
            final_point = final_point + self.frames_chunk   
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        sliced_dim = (self.dim[0], self.dim[1], self.y_VolSlice_end - self.y_VolSlice_start)
        X = np.empty((1, *sliced_dim, 4))
        y = np.zeros((1,*sliced_dim,4))
        
        random.seed(time.time())

        for i, file_paths in enumerate(list_IDs_temp):
            image = self.load_nifti(file_paths)
                

            X[0] = image
            
            
            # Set the label
            seg = nib.load(self.labels[i]).get_fdata().astype(np.float16)[:,:,self.y_VolSlice_start:self.y_VolSlice_end]
            seg[seg==4] = 3

            re= np.expand_dims(seg, axis=0)

            re = tf.image.resize(re,(156,156))
            re = re[0]
            seg = tf.cast(re, tf.int32)
            print(seg.shape)
            mask = tf.one_hot(seg, 4);
            
            
            y[0] = mask
            
        X = X.astype(np.float16)
        y = y.astype(np.float16)
        
        return X[0], y[0]

    def load_nifti(self, file_paths):
        i = 0
        temp = nib.load(inputpath[i][0]).get_fdata().astype(np.float16)[:,:,self.y_VolSlice_start:self.y_VolSlice_end]
        re= np.expand_dims(temp, axis=0)
        re = tf.image.resize(re,(156,156))
        temp = re[0]
        
        temp = np.expand_dims(temp, axis=3)
        for channels in range(1,len(file_paths)):
            tempo = nib.load(file_paths[channels]).get_fdata().astype(np.float16)[:,:,self.y_VolSlice_start:self.y_VolSlice_end]
            re= np.expand_dims(tempo, axis=0)
            re = tf.image.resize(re,(156,156))
            tempo = re[0]
            tempo = np.expand_dims(tempo, axis=3)
            temp = np.concatenate((temp,tempo),axis=3)
        
        return temp


import tensorflow.keras as keras
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
#set up var
x_VolSlice_start = 10
x_VolSlice_end = 150

y_VolSlice_start = 10
y_VolSlice_end = 150

# x_train = inputimg[:1126].copy()
# y_train = outputimg[:1126].copy()
# x_train.shape[0] == y_train.shape[0]
# x_eval = inputimg[1126:1189].copy()
# y_eval = outputimg[1126:1189].copy()
# print(x_eval.shape, y_eval.shape)
# print(x_eval.shape[0] == y_eval.shape[0])
# x_test = inputimg[1189:].copy()
# y_test = outputimg[1189:].copy()

# print("train target:",y_train.shape,"                test target:",y_test.shape)
# print(f"train x{x_test.shape}","                test x:", x_test.shape)

# #shuffle data
# np.random.seed(0)
# np.random.shuffle(x_train)
# np.random.shuffle(y_train)
# np.random.shuffle(x_eval)
# np.random.shuffle(y_eval)

# np.random.shuffle(x_test)
# np.random.shuffle(y_test)

# y_train = y_train.astype(int)
# y_eval = y_eval.astype(int)
# y_test = y_test.astype(int)


# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import numpy as np

# input_shape = (240, 240, 4)
# num_classes = 4

# # Data preparation functions
# #why my last code didnt work:
# #the labels in the output goes 0,1,2,4????????
# #yeah 0,1,2,4 not 0,1,2,3 but 0,1,2,4
# #no wonder i am now depressed

# #side note: np.unique(tells us all unique element)
# def correct_labels(y):
#     y[y == 4] = 3
#     return y
# #normalize data
# def normalize_data(x):
#     return (x - np.min(x)) / (np.max(x) - np.min(x))

# # Apply corrections to your data
# x_train = normalize_data(x_train)
# x_eval = normalize_data(x_eval)
# x_test = normalize_data(x_test)

# y_train = correct_labels(y_train)
# y_eval = correct_labels(y_eval)
# y_test = correct_labels(y_test)

# Rest of your code remains the same
# from keras import layers
# def unet_model(input_shape, num_classes):
#     inputs = keras.Input(input_shape)
    
#     # Encoder (downsampling)
#     conv1 = conv_block(inputs, 64)
#     pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
#     conv2 = conv_block(pool1, 128)
#     pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
#     conv3 = conv_block(pool2, 256)
#     pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    
#     # Bridge
#     conv4 = conv_block(pool3, 512)
    
#     # Decoder (upsampling)
#     up5 = layers.UpSampling2D(size=(2, 2))(conv4)
#     up5 = layers.concatenate([up5, conv3])
#     conv5 = conv_block(up5, 256)
    
#     up6 = layers.UpSampling2D(size=(2, 2))(conv5)
#     up6 = layers.concatenate([up6, conv2])
#     conv6 = conv_block(up6, 128)
    
#     up7 = layers.UpSampling2D(size=(2, 2))(conv6)
#     up7 = layers.concatenate([up7, conv1])
#     conv7 = conv_block(up7, 64)
    
#     # Output layer
#     outputs = layers.Conv2D(num_classes, 1, activation='softmax')(conv7)
    
#     return keras.Model(inputs=inputs, outputs=outputs)
    

# def conv_block(input_tensor, num_filters):
#     x = layers.Conv2D(num_filters, 3, padding='same')(input_tensor)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('relu')(x)
#     x = layers.Dropout(0.1)(x)
    
#     x = layers.Conv2D(num_filters, 3, padding='same')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('relu')(x)
#     return x

# model = unet_model(input_shape, num_classes)

# optimizer = keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0)
# model.compile(optimizer=optimizer, 
#               loss='categorical_crossentropy', 
#               metrics=['accuracy'])

# # Print model summary
# model.summary()
#




#-----------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------model making--------------------------------------------------------------------------------

#note:
# conv3d need a 5d tensor (height,width,depth,channels)


import tensorflow as tf
from tensorflow.keras import layers, Model

def conv_block_3d(input_tensor, num_filters):
    x = layers.Conv3D(num_filters, 3, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv3D(num_filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def unet_3d(input_shape=(156, 156, 140, 4), num_classes=4):
    inputs = layers.Input(input_shape, dtype='float16')
    
    # Encoder (downsampling)
    conv1 = conv_block_3d(inputs, 32)
    pool1 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    
    conv2 = conv_block_3d(pool1, 64)
    pool2 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    
    conv3 = conv_block_3d(pool2, 128)
    pool3 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    
    conv4 = conv_block_3d(pool3, 256)
    pool4 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv4)
    
    # Bridge
    conv5 = conv_block_3d(pool4, 512)
    
    # Decoder (upsampling)
    up6 = layers.Conv3DTranspose(256, 2, strides=(2, 2, 2), padding='same')(conv5)
    up6 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv4.shape[1] - x.shape[1]], [0, conv4.shape[2] - x.shape[2]], [0, conv4.shape[3] - x.shape[3]], [0, 0]]))(up6)
    concat6 = layers.Concatenate()([up6, conv4])
    conv6 = conv_block_3d(concat6, 256)
    
    up7 = layers.Conv3DTranspose(128, 2, strides=(2, 2, 2), padding='same')(conv6)
    up7 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv3.shape[1] - x.shape[1]], [0, conv3.shape[2] - x.shape[2]], [0, conv3.shape[3] - x.shape[3]], [0, 0]]))(up7)
    concat7 = layers.Concatenate()([up7, conv3])
    conv7 = conv_block_3d(concat7, 128)
    
    up8 = layers.Conv3DTranspose(64, 2, strides=(2, 2, 2), padding='same')(conv7)
    up8 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv2.shape[1] - x.shape[1]], [0, conv2.shape[2] - x.shape[2]], [0, conv2.shape[3] - x.shape[3]], [0, 0]]))(up8)
    concat8 = layers.Concatenate()([up8, conv2])
    conv8 = conv_block_3d(concat8, 64)
    
    up9 = layers.Conv3DTranspose(32, 2, strides=(2, 2, 2), padding='same')(conv8)
    up9 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv1.shape[1] - x.shape[1]], [0, conv1.shape[2] - x.shape[2]], [0, conv1.shape[3] - x.shape[3]], [0, 0]]))(up9)
    concat9 = layers.Concatenate()([up9, conv1])
    conv9 = conv_block_3d(concat9, 32)
    
    outputs = layers.Conv3D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define Dice coefficient
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# Define Dice loss
def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

# Combine Dice loss and categorical crossentropy
def combined_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + tf.keras.losses.categorical_crossentropy(y_true, y_pred)

tf.debugging.set_log_device_placement(True)
import tensorflow as tf
tf.keras.backend.clear_session()

strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


print("start")
traingen = DataGenerator(parition["train"],label["train"],5,(156,156,140),1,4,True,y_VolSlice_start=10,y_VolSlice_end=150)
validationgen = DataGenerator(parition["validation"],label["validation"],5,(156,156,140),1,4,True,y_VolSlice_start=10,y_VolSlice_end=150)
# Create the model
with strategy.scope():
    model = unet_3d()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=combined_loss,
              metrics=[dice_coefficient, 'accuracy',tf.keras.metrics.MeanIoU(4)])
tf.debugging.set_log_device_placement(True)
print("here")

model.summary()

model.fit(
traingen,
validation_data=validationgen,
epochs=80,verbose=1,batch_size=5)


Done
['/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_flair.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_seg.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_t1.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_t1ce.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_t2.nii.gz']


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 145, 4)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_30 (Conv3D)  │ (None, 240, 240,  │      3,488 │ input_layer_4[0]… │
│                     │ 145, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 240, 240,  │        128 │ conv3d_30[0][0]   │
│ (BatchNormalizatio… │ 145, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_42       │ (None, 240, 240,  │          0 │ batch_normalizat… │
│ (Activation)        │ 145, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_31 (Conv3D)  │ (None, 240, 240,  │     27,680 │ activation_42[0]… │
│                     │ 145, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 240, 240,  │        128 │ conv3d_31[0][0]   │
│ (BatchNormalizatio… │ 145, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_43       │ (None, 240, 240,  │          0 │ batch_normalizat… │
│ (Activation)        │ 145, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_8     │ (None, 120, 120,  │          0 │ activation_43[0]… │
│ (MaxPooling3D)      │ 72, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_32 (Conv3D)  │ (None, 120, 120,  │     55,360 │ max_pooling3d_8[… │
│                     │ 72, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │        256 │ conv3d_32[0][0]   │
│ (BatchNormalizatio… │ 72, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_44       │ (None, 120, 120,  │          0 │ batch_normalizat… │
│ (Activation)        │ 72, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_33 (Conv3D)  │ (None, 120, 120,  │    110,656 │ activation_44[0]… │
│                     │ 72, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 120, 120,  │        256 │ conv3d_33[0][0]   │
│ (BatchNormalizatio… │ 72, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_45       │ (None, 120, 120,  │          0 │ batch_normalizat… │
│ (Activation)        │ 72, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_9     │ (None, 60, 60,    │          0 │ activation_45[0]… │
│ (MaxPooling3D)      │ 36, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_34 (Conv3D)  │ (None, 60, 60,    │    221,312 │ max_pooling3d_9[… │
│                     │ 36, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 60,    │        512 │ conv3d_34[0][0] 

 Total params: 22,589,796 (86.17 MB)

 Trainable params: 22,583,908 (86.15 MB)

 Non-trainable params: 5,888 (23.00 KB)

In [20]:
#set up var
x_VolSlice_start = 5
x_VolSlice_end = 150

y_VolSlice_start = 6
y_VolSlice_end = 150

#testing
x = nib.load(inputpath[0][1]).get_fdata()[:,:,x_VolSlice_start:x_VolSlice_end]
y = nib.load(outputpath[0]).get_fdata()[:,:,y_VolSlice_start:y_VolSlice_end]
print(x.shape,y.shape)

(240, 240, 145) (240, 240, 144)


In [1]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# Define Dice loss
def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

# Combine Dice loss and categorical crossentropy
def combined_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + tf.keras.losses.categorical_crossentropy(y_true, y_pred)

In [22]:

#prep output and input into array
import numpy as np
import nibabel as nib
import tensorflow as tf
import os

def process_and_save_output(outputpath, x_VolSlice_start, x_VolSlice_end, batch_size=1, output_file='output_dataset.npy'):
    total_slices = 0
    for i in range(0, len(outputpath), batch_size):
        batch_paths = outputpath[i:i+batch_size]
        for j, path in enumerate(batch_paths):
            img = nib.load(path).dataobj[:,:,x_VolSlice_start:x_VolSlice_end]
            img = np.array(img, dtype=np.float32)
            
            # Process each slice separately
            processed_slices = []
            for slice_idx in range(img.shape[2]):
                slice_img = img[:,:,slice_idx]
                slice_img = np.expand_dims(slice_img, axis=2)  # Add channel dimension
                slice_img = tf.image.resize(slice_img, (256, 256)).numpy()
                processed_slices.append(slice_img[:,:,0])  # Remove channel dimension
            
            img = np.stack(processed_slices, axis=0)  # Stack all processed slices
            
            if i == 0 and j == 0:
                np.save(output_file, img)
            else:
                with open(output_file, 'ab') as f:
                    np.save(f, img)
            
            total_slices += img.shape[0]
            del img, processed_slices
        print(f"Processed and saved batch {i//batch_size + 1}/{len(outputpath)//batch_size + 1}")
    
    print(f"Total slices processed: {total_slices}")
    return total_slices

# The rest of the code remains the same
def process_and_save_input(inputpath, y_VolSlice_start, y_VolSlice_end, batch_size=1, output_file='input_dataset.npy'):
    total_samples = 0
    for i in range(0, len(inputpath), batch_size):
        batch_paths = inputpath[i:i+batch_size]
        for paths in batch_paths:
            img = nib.load(paths[0]).dataobj[:,:,y_VolSlice_start:y_VolSlice_end]
            img = np.array(img, dtype=np.float32)
            for path in paths[1:]:
                channel = nib.load(path).dataobj[:,:,y_VolSlice_start:y_VolSlice_end]
                channel = np.array(channel, dtype=np.float32)
                img = np.dstack((img, channel))
                del channel
            img = np.expand_dims(img, axis=0)
            
            if i == 0 and total_samples == 0:
                np.save(output_file, img)
            else:
                with open(output_file, 'ab') as f:
                    np.save(f, img)
            
            total_samples += 1
            del img
        print(f"Processed and saved input batch {i//batch_size + 1}/{len(inputpath)//batch_size + 1}")
    
    print(f"Total input samples processed: {total_samples}")
    return total_samples

# Main execution
batch_size = 1  # Process one file at a time
output_slices = process_and_save_output(outputpath, x_VolSlice_start, x_VolSlice_end, batch_size)
input_samples = process_and_save_input(inputpath, y_VolSlice_start, y_VolSlice_end, batch_size)

print(f"Output shape: ({output_slices}, 256, 256)")
print(f"Input shape: ({input_samples}, {inputpath[0][0].shape[0]}, {inputpath[0][0].shape[1]}, {len(inputpath[0])})")

#PLEASE FOR THE LOVE OF GOD
#LET THIS WORK PLEASE

#now it runs on 1 file at once nice
# and it saves on a npy on a disk instead of in memory


1


2024-06-27 15:16:17.929575: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (mklcpu) ran out of memory trying to allocate 8.50GiB (rounded to 9122611200)requested by op ResizeBilinear
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-06-27 15:16:17.929909: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for mklcpu
2024-06-27 15:16:17.930286: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-06-27 15:16:17.930322: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-06-27 15:16:17.930325: I external/local_tsl/tsl/f

ResourceExhaustedError: {{function_node __wrapped__ResizeBilinear_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[240,256,256,145] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:ResizeBilinear] name: 

(240, 240, 144)

In [ ]:
with open('history.txt', 'w') as f:
    data = f.write()